In [ ]:
import os
import random
import evaluate
import mlflow

import numpy as np
import pandas as pd
import torch

from datasets import Dataset, DatasetDict
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)
from transformers import (
    AutoTokenizer,
    BertForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    pipeline
)


In [45]:

# disable WandB defaults
os.environ['WANDB_DISABLED'] = 'true'


# a seed for reproducibility
SEED = 42
# set seed
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)

# check for GPU device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device available:', device) 



Device available: cpu


In [46]:
df = pd.read_csv('../data/ml_data.csv')

In [47]:
df['price_class'] = pd.qcut(df['price'], q=10, labels=range(0, 10))

In [48]:
df_texts = df[['text', 'price_class']]

In [49]:
df_texts['text'] = df_texts['text'].astype(str)

C:\Users\user\AppData\Local\Temp\ipykernel_69964\3160330217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_texts['text'] = df_texts['text'].astype(str)


In [50]:
df_texts

,text,price_class
0,"В продаже Skoda Kodiaq, полный привод.Оригинал...",9
1,Шевроле Круз 2010г 1.6 механика.Двигатель рабо...,3
2,"На ходу, на учёте, 1.6 8клоп, колеса зима шипы...",0
3,"Ваз-2107, в отличном техническом состоянии, вс...",3
4,Майбах S400 4matic в полном рестайлинге внаруж...,9
...,...,...
40446,"Автомобиль в идеальном состоянии, пробег, кузо...",9
40447,Продам ваз 2115.(Родной пробег 135.000).Птс ор...,1
40448,Машина на полном ходу все расходники поменяны....,4
40449,Только продажа цена снижена за срочность.,6


In [51]:
dataset = Dataset.from_pandas(df_texts)

In [52]:
splited_dataset = dataset.train_test_split(0.2)

In [53]:
train_dataset = splited_dataset['train']

In [54]:
test_dataset = splited_dataset['test'].train_test_split(0.5)['train']
val_dataset = splited_dataset['test'].train_test_split(0.5)['test']

In [55]:
full_dataset = DatasetDict({'train': train_dataset, 'test': test_dataset, 'val': val_dataset})

In [56]:
full_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'price_class'],
        num_rows: 32360
    })
    test: Dataset({
        features: ['text', 'price_class'],
        num_rows: 4045
    })
    val: Dataset({
        features: ['text', 'price_class'],
        num_rows: 4046
    })
})

In [57]:
checkpoint = 'cointegrated/rubert-tiny2'
# bert tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# data collator for dynamic padding as per batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [58]:
# cache a pre-trained BERT model for two-class classification
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=10)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
# define a tokenize function
def Tokenize_function(example):
    return tokenizer(example['text'], truncation=True)

In [60]:
full_dataset['train'][0]

{'text': 'Была проведена полная диагностика, в результате которой нарушений не выявлено.Техническое состояние отличное.Птс оригинал.',
 'price_class': 7}

In [61]:
# tokenize entire data
tokenized_data = full_dataset.map(Tokenize_function, batched=True)

Map:   0%|          | 0/32360 [00:00<?, ? examples/s]

Map: 100%|██████████| 4046/4046 [00:00<00:00, 11125.18 examples/s]


In [62]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['text', 'price_class', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32360
    })
    test: Dataset({
        features: ['text', 'price_class', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4045
    })
    val: Dataset({
        features: ['text', 'price_class', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4046
    })
})

In [63]:
tokenized_data = tokenized_data.remove_columns(['text'])
tokenized_data = tokenized_data.rename_column('price_class','labels')
tokenized_data.with_format('pt')

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32360
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4045
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4046
    })
})

In [64]:
# use the pre-built metrics 
def compute_metrics(eval_preds):
    f1_metric = evaluate.load('f1')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return f1_metric.compute(predictions=predictions, references=labels, average='macro')

In [65]:
def calculate_metrics(y_pred, y_test, average):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average=average)
    recall = recall_score(y_test, y_pred, average=average)
    f1 = f1_score(y_test, y_pred, average=average)

    return accuracy, precision, recall, f1

In [66]:
y_test = test_dataset['price_class']

In [71]:
mlflow.set_tracking_uri('../mlruns')
mlflow.set_experiment('Text classificator (10)')

with mlflow.start_run():
    training_params = {
        'output_dir': 'bert-finetuning',
        'eval_strategy': 'epoch',
        'num_train_epochs': 3,
        'learning_rate': 5e-5,
        'weight_decay': 0.005,
        'per_device_train_batch_size': 8,
        'per_device_eval_batch_size': 8,
        'report_to': 'none',
    }

    model_config = {'batch_size': 8}

    training_args = TrainingArguments(**training_params)
    
    mlflow.log_params(training_params)

    trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_data['train'],
        eval_dataset=tokenized_data['val'],
        data_collator=data_collator,
        processing_class =tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    tuned_pipeline = pipeline(
        task='text-classification',
        model=trainer.model,
        batch_size=8,
        tokenizer=tokenizer,
        device='cpu',
    )
    predictions = trainer.predict(tokenized_data['test'])
    y_pred = np.argmax(predictions.predictions, axis=1) 

    accuracy, precision, recall, f1 = calculate_metrics(y_pred, y_test, 'macro')
    
    print('Rubert tiny2 model')
    print(f'  Accuracy: {accuracy}')
    print(f'  Precision: {precision}')
    print(f'  Recall: {recall}')
    print(f'  F1: {f1}')

    mlflow.log_metric('Accuracy', accuracy)
    mlflow.log_metric('Precision', precision)
    mlflow.log_metric('Recall', recall)
    mlflow.log_metric('F1', f1)

    
    # Логирование модели с примером входных данных
    mlflow.transformers.log_model(
        transformers_model=tuned_pipeline,
        artifact_path='bert-finetuning',
        model_config=model_config,
    )

  4%|▍         | 500/12135 [03:27<1:16:24,  2.54it/s]

{'loss': 1.5946, 'grad_norm': 17.380924224853516, 'learning_rate': 4.793984342810054e-05, 'epoch': 0.12}


  8%|▊         | 1000/12135 [06:37<1:11:59,  2.58it/s]

{'loss': 1.5732, 'grad_norm': 30.89073944091797, 'learning_rate': 4.587968685620108e-05, 'epoch': 0.25}


 12%|█▏        | 1500/12135 [09:42<1:00:08,  2.95it/s]

...

100%|██████████| 12135/12135 [1:37:42<00:00,  2.62it/s]


















































































































































































                                                       

100%|██████████| 12135/12135 [1:38:09<00:00,  2.62it/s]


100%|██████████| 12135/12135 [1:38:09<00:00,  2.06it/s]


{'eval_loss': 1.933677315711975, 'eval_f1': 0.3184332617611575, 'eval_runtime': 26.017, 'eval_samples_per_second': 155.514, 'eval_steps_per_second': 19.449, 'epoch': 3.0}
{'train_runtime': 5889.7328, 'train_samples_per_second': 16.483, 'train_steps_per_second': 2.06, 'train_loss': 1.4205874008851431, 'epoch': 3.0}


100%|██████████| 506/506 [00:24<00:00, 20.42it/s]


Rubert tiny2 model
  Accuracy: 0.3100123609394314
  Precision: 0.310259412496028
  Recall: 0.3113448643376934
  F1: 0.30994959600077765


2024/11/14 16:54:16 WARNING mlflow.utils.environment: On Windows, timeout is not supported for model requirement inference. Therefore, the operation is not bound by a timeout and may hang indefinitely. If it hangs, please consider specifying the signature manually.
